<a href="https://colab.research.google.com/github/4entertainment/nlp_works/blob/main/sent-analysis-wthrr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis**

# Loading the dataset and making it ready for use

In [41]:
import numpy as np
import pandas as pd

In [42]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [116]:
dataset = pd.read_csv('/content/sample_data/hepsiburada.csv')
# reading the dataset

In [117]:
dataset
# looking at the dataset

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
6214,1,Fiyat/Performans olarak bakıldığında çok güzel...
6215,1,Ölü bilgisayarı dirilten bir alet alın takın 5...
6216,1,harika
6217,1,Harika ötesi bir hız ve kalite çok memnun kald...


In [118]:
target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()
# getting data ratings and their review tags.

In [119]:
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]
# seperating the dataset (data ratings and review tags) as train and test dataset parts.

In [120]:
x_train[500]
# looking at the 500.index data

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım.bu yüzden tereddütsüz aldım . alacak olanlara öneririm'

In [121]:
x_train[800]
# looking at the 800.index data

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [122]:
y_train[800]
# looking at the 800.index data's review tag (1=>positive)

1

# Tokenization

In [123]:
# tokenization the reviews.
num_words = 10000 # getting the 10000 words in dataset.
tokenizer = Tokenizer(num_words=num_words)


In [124]:
tokenizer.fit_on_texts(data)
# tokenization the given data(reviews' initialized name was"data")

In [125]:
# looking at the all tokens sorted by priority of most used
tokenizer.word_index

{'bir': 1,
 'çok': 2,
 've': 3,
 'ürün': 4,
 'bu': 5,
 'iyi': 6,
 'için': 7,
 'tavsiye': 8,
 'güzel': 9,
 'ederim': 10,
 'gayet': 11,
 'hızlı': 12,
 'daha': 13,
 'da': 14,
 'aldım': 15,
 'ama': 16,
 'de': 17,
 'kullanışlı': 18,
 'yok': 19,
 'teşekkürler': 20,
 'ürünü': 21,
 'küçük': 22,
 'elime': 23,
 'uygun': 24,
 'mouse': 25,
 'kaliteli': 26,
 'gibi': 27,
 'olarak': 28,
 'fiyat': 29,
 'ile': 30,
 'fiyata': 31,
 'en': 32,
 'kargo': 33,
 'göre': 34,
 '2': 35,
 'kullanıyorum': 36,
 'gerçekten': 37,
 'var': 38,
 'kadar': 39,
 'hepsiburada': 40,
 'kesinlikle': 41,
 'her': 42,
 'hiç': 43,
 'ben': 44,
 'usb': 45,
 'hızı': 46,
 'hem': 47,
 'ulaştı': 48,
 '1': 49,
 '3': 50,
 'şık': 51,
 'geldi': 52,
 'olması': 53,
 'cok': 54,
 'mükemmel': 55,
 'biraz': 56,
 'bence': 57,
 'tek': 58,
 'pil': 59,
 'tam': 60,
 'performans': 61,
 'değil': 62,
 'bi': 63,
 'fiyatı': 64,
 'oldukça': 65,
 'süper': 66,
 'ayrıca': 67,
 'önce': 68,
 'memnunum': 69,
 'gün': 70,
 'oldu': 71,
 'gb': 72,
 'sağlam': 73,
 'çal

In [126]:
# tokenization of the train set
x_train_tokens = tokenizer.texts_to_sequences(x_train)

In [127]:
# looking at the 800.index data
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [128]:
# looking at the 800.index data as tokenized form
print(x_train_tokens[800])

[21, 344, 50, 257, 39, 71, 3916, 36, 8527, 297, 1141, 5176, 3, 44, 139, 166, 727, 256, 8528, 8529, 2, 1037, 580, 39, 1, 296, 243, 3, 2, 69, 152, 407, 8, 10]


In [129]:
# tokenizaton of the test set
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [130]:
'''
we always work with data of certain sizes. all data should be made to the same size.
we will specify a size. if the size of the data is smaller than this specified size,
it will fill in the missing places by assigning "0"
'''

# getting every review's token size then writing the token size into a list.
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
# for the easy calculations, we transform the list to numpy array.
# in this way of numpy library, we can get token or token size informations easily
num_tokens = np.array(num_tokens)

In [131]:
# calculation the average number of tokens found in a review (in all dataset)
np.mean(num_tokens)

20.498633220775044

In [132]:
# looking at the max number of tokens found in a review (in all dataset)
np.max(num_tokens)

231

In [133]:
# founding the maximum sized review in all dataset with argmax function
np.argmax(num_tokens)

5711

In [134]:
x_train[670]

'Ürünü biraz önce teslim aldım ve hemen denedim gerçekten kaliteli ve verilen paraya layık bir ürün almak isteyenlere hiç tereddütsüz öneririm...'

In [135]:
# we should give the spesific sized datas to network. to equal all data(review) size; we pedding the data's tokens.
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) # convertion to integer
max_tokens
# we will give data with 52 tokens to our network

52

In [136]:
np.sum(num_tokens < max_tokens) / len(num_tokens)
# looking at the "these 52 tokens cover how many of the data"
# 95% have less than 52 tokens
# 5% have more than 52 tokens
# we will add or delete tokens from each data to bring the token count of each data to 52
# data loss occurs in token deletion. however, this loss will be ignored because the dataset is large enough.

0.9588358256954495

In [137]:
# with this padding, every review(data) has 52 tokens.
# for train set:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)

In [138]:
# with this padding, every review(data) has 52 tokens.
# for test set:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [139]:
x_train_pad.shape

(4975, 52)

In [140]:
x_test_pad.shape

(1244, 52)

In [141]:
np.array(x_train_tokens[800])

array([  21,  344,   50,  257,   39,   71, 3916,   36, 8527,  297, 1141,
       5176,    3,   44,  139,  166,  727,  256, 8528, 8529,    2, 1037,
        580,   39,    1,  296,  243,    3,    2,   69,  152,  407,    8,
         10])

In [142]:
x_train_pad[800]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   21,  344,   50,  257,
         39,   71, 3916,   36, 8527,  297, 1141, 5176,    3,   44,  139,
        166,  727,  256, 8528, 8529,    2, 1037,  580,   39,    1,  296,
        243,    3,    2,   69,  152,  407,    8,   10], dtype=int32)

In [144]:
# with this code, we can give the number to find the releated word to given number
idx = tokenizer.word_index # getting the word index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [146]:
# creating a function to use the previous expression on a text:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words) # with this, the words will be written one after the other with a space between them
    return text

In [149]:
x_train[800]
# pretokenized 800.indexed word

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [150]:
tokens_to_string(x_train_tokens[800])
# with x_train_tokens[], we get tokens of 800.indexed word as numeralically

'ürünü alalı 3 hafta kadar oldu aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim'

# Model Creating & Training

Data has been loaded so far. It was divided into two parts as training and testing. tokenization has been carried out. The model is now ready to be created.

First the embedding matrix will be created. Training will be held with a 3-storey GRU. There will be a single neuron at the end of the GRU. If the value produced by this last neuron is close to 1, the result will be called positive, and if it is close to 0, the result will be called negative.

**Creating the model:**

In [152]:
model = Sequential()
# creating the model with sequential quality

In [155]:
embedding_size = 50
# definition of embedding matrix size
# will create a '50' long vector size corresponding to each word

In [158]:
# creating embedding layer in keras
# we use randomly created vectors. we dont use glove or word2vec trained vectors.
# adding the embedding layer to our model with model.add function:
model.add(Embedding(input_dim=num_words,        # num_words: 10000.
                    output_dim=embedding_size,  # embedding_size: 50. we get 10000x50 sized embedding matrix.
                                                # we have 10000 words. and every word has 50 sized vector. vectors will create with randomly
                    input_length=max_tokens,    # input length was determined as '52' sized with max_tokens
                    name='embedding_layer1'))    #

In [167]:
model = Sequential()
embedding_size = 50
model.add(Embedding(input_dim=num_words, output_dim=embedding_size, input_length=max_tokens, name='embedding_layer1'))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))
'''
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))
'''



"\nmodel.add(GRU(units=16, return_sequences=True))\nmodel.add(GRU(units=8, return_sequences=True))\nmodel.add(GRU(units=4))\nmodel.add(Dense(1, activation='sigmoid'))\n"

In [181]:
from tensorflow.keras.layers import Embedding, GRU, Dense
model.add(Embedding(input_dim=num_words, output_dim=embedding_size, input_length=max_tokens, name='embedding_layer2'))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

# Optimizer tanımlama
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Modeli derleme
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# Şimdi model eğitim için hazır

ValueError: ignored

In [176]:
 import tensorflow as tf
 #optimizer = Adam(learning_rate=1e-3)
 optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [178]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


ValueError: ignored

In [ ]:
model.summary()

In [ ]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=256)

In [ ]:
result = model.evaluate(x_test_pad, y_test)

In [ ]:
result[1]

In [30]:
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

NameError: ignored

In [ ]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [ ]:
cls_true = np.array(y_test[0:1000])

In [ ]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [ ]:
len(incorrect)

In [ ]:
idx = incorrect[0]
idx

In [ ]:
text = x_test[idx]
text

In [ ]:
y_pred[idx]

In [ ]:
cls_true[idx]

In [ ]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [ ]:
tokens = tokenizer.texts_to_sequences(texts)

In [ ]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

In [31]:
model.predict(tokens_pad)

NameError: ignored